In [1]:
import nltk
import pandas as pd
import numpy as np
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer


import warnings
warnings.filterwarnings('ignore')

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob, Word
from nltk.stem import PorterStemmer

from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn import datasets

from sklearn.model_selection import  RandomizedSearchCV,GridSearchCV

In [57]:
from sklearn.metrics import accuracy_score,precision_score, auc,recall_score, precision_recall_curve, fbeta_score, confusion_matrix, classification_report, make_scorer,f1_score,roc_auc_score

def model_eval(mdl_local, X_local, y_local, bta =1):
    y_pred = mdl_local.predict(X_local)
    
    accuracy= accuracy_score(y_local, y_pred)
    f1= fbeta_score(y_local, y_pred, beta= bta, average='micro')
    precision= precision_score(y_local, y_pred, average='micro')
    recall= recall_score(y_local, y_pred, average='micro')
    print("Accurcy :",accuracy)
    print(f"F({bta}):",f1)
    print("Precision :",precision)
    print("Recall :",recall)
    
    cm_results = [accuracy, precision, recall, f1]
    return cm_results

In [3]:
df = pd.read_csv('DataAfterMask.csv', index_col=0)
df

,headline,clickbait,topices,popular_topices
0,get bings,1,0,Entertainment
1,tv female friend group belong,1,0,Entertainment
2,new star war force awakens trailer give chill,1,0,Entertainment
3,vine new york celebrity big brother fucking pe...,1,0,Entertainment
4,couple stunning photo shoot baby learning inop...,1,2,Lifestyle
...,...,...,...,...
15994,mini sisterhood traveling pant reunion,1,1,Politics
15995,dog thankful best friend,1,2,Lifestyle
15996,people proving dick big dropping condom head,1,2,Lifestyle
15997,im atheist im,1,0,Entertainment


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15999 entries, 0 to 15998
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   headline         15997 non-null  object
 1   clickbait        15999 non-null  int64 
 2   topices          15999 non-null  int64 
 3   popular_topices  15999 non-null  object
dtypes: int64(2), object(2)
memory usage: 625.0+ KB


In [5]:
df.isna().sum()

headline           2
clickbait          0
topices            0
popular_topices    0
dtype: int64

In [6]:
df.dropna(inplace=True)

# determine lable and target

In [8]:
x=df.headline
y=df.topices

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# count vectorizer

In [10]:
# The first document-term matrix has default Count Vectorizer values - counts of unigrams
from sklearn.feature_extraction.text import CountVectorizer

cv1 = CountVectorizer(stop_words='english')

X_train_cv1 = cv1.fit_transform(X_train)
X_test_cv1  = cv1.transform(X_test)

cv2 = CountVectorizer(ngram_range=(1,2), binary=True, stop_words='english')

X_train_cv2 = cv2.fit_transform(X_train)
X_test_cv2  = cv2.transform(X_test)


### logistic regression

In [58]:
from sklearn.linear_model import LogisticRegression 
lr = LogisticRegression()
lr.fit(X_train_cv1, y_train)
y_pred_cv1 = lr.predict(X_test_cv1)
print("Train Logistic Regression :",lr.score(X_train_cv1, y_train))
me1=model_eval(lr,X_test_cv1,y_pred_cv1)
me1

Train Logistic Regression : 0.9669554344913817
Accurcy : 1.0
F(1): 1.0
Precision : 1.0
Recall : 1.0


[1.0, 1.0, 1.0, 1.0]

In [59]:
lr = LogisticRegression()
lr.fit(X_train_cv2, y_train)
y_pred_cv2 = lr.predict(X_test_cv2)
print("Train Logistic Regression :",lr.score(X_train_cv2, y_train))
me2=model_eval(lr,X_test_cv2,y_pred_cv2)


Train Logistic Regression : 0.9992855229079217
Accurcy : 1.0
F(1): 1.0
Precision : 1.0
Recall : 1.0


### Naive Bayes-Multinomial

In [60]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train_cv1, y_train)
y_pred_cv1_nb = mnb.predict(X_test_cv1)
me3=model_eval(mnb,X_test_cv1,y_pred_cv1_nb)

Accurcy : 1.0
F(1): 1.0
Precision : 1.0
Recall : 1.0


In [61]:
mnb = MultinomialNB()
mnb.fit(X_train_cv2, y_train)
y_pred_cv2_nb = mnb.predict(X_test_cv2)
me4=model_eval(mnb,X_test_cv2,y_pred_cv2_nb)

Accurcy : 1.0
F(1): 1.0
Precision : 1.0
Recall : 1.0


### Naive Bayes-Bernoulli

In [62]:
# Fit the second Naive Bayes model
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
bnb.fit(X_train_cv1, y_train)
y_pred_cv1_nb = bnb.predict(X_test_cv1)
me5=model_eval(bnb,X_test_cv1,y_pred_cv1_nb)

Accurcy : 1.0
F(1): 1.0
Precision : 1.0
Recall : 1.0


In [63]:
bnb = BernoulliNB()
bnb.fit(X_train_cv2, y_train)
y_pred_cv2_nb = bnb.predict(X_test_cv2)
me6=model_eval(bnb,X_test_cv2,y_pred_cv2_nb)

Accurcy : 1.0
F(1): 1.0
Precision : 1.0
Recall : 1.0


# TF-IDF

In [64]:
# Create TF-IDF versions of the Count Vectorizers created earlier in the exercise
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf1 = TfidfVectorizer(stop_words='english')
X_train_tfidf1 = tfidf1.fit_transform(X_train)
X_test_tfidf1  = tfidf1.transform(X_test)

tfidf2 = TfidfVectorizer(ngram_range=(1,2), binary=True, stop_words='english')
X_train_tfidf2 = tfidf2.fit_transform(X_train)
X_test_tfidf2  = tfidf2.transform(X_test)

### logistic regression

In [65]:
# Fit the first logistic regression on the TF-IDF data
lr.fit(X_train_tfidf1, y_train)
y_pred_tfidf1_lr = lr.predict(X_test_tfidf1)
me7=model_eval(lr,X_test_tfidf1,y_pred_tfidf1_lr)

Accurcy : 1.0
F(1): 1.0
Precision : 1.0
Recall : 1.0


In [66]:
# Fit the first logistic regression on the TF-IDF data
lr.fit(X_train_tfidf2, y_train)
y_pred_tfidf2_lr = lr.predict(X_test_tfidf2)
me8=model_eval(lr,X_test_tfidf2,y_pred_tfidf2_lr)

Accurcy : 1.0
F(1): 1.0
Precision : 1.0
Recall : 1.0


### Naive Bayes-Multinomial

In [67]:
mnb = MultinomialNB()
mnb.fit(X_train_tfidf1, y_train)
y_pred_tfidf1_nb = mnb.predict(X_test_tfidf1)

me9=model_eval(mnb,X_test_tfidf1,y_pred_tfidf1_nb)

Accurcy : 1.0
F(1): 1.0
Precision : 1.0
Recall : 1.0


In [68]:
mnb = MultinomialNB()
mnb.fit(X_train_tfidf2, y_train)
y_pred_tfidf2_nb = mnb.predict(X_test_tfidf2)

me10=model_eval(mnb,X_test_tfidf2,y_pred_tfidf2_nb)

Accurcy : 1.0
F(1): 1.0
Precision : 1.0
Recall : 1.0


### Naive Bayes-Bernoulli

In [69]:
# Fit the second Naive Bayes model
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
bnb.fit(X_train_tfidf1, y_train)
y_pred_tfidf_nb = bnb.predict(X_test_tfidf1)
me11=model_eval(bnb,X_test_tfidf1,y_pred_tfidf_nb)

Accurcy : 1.0
F(1): 1.0
Precision : 1.0
Recall : 1.0


In [70]:
bnb = BernoulliNB()
bnb.fit(X_train_tfidf2, y_train)
y_pred_tfidf2_nb = bnb.predict(X_test_tfidf2)
me12=model_eval(bnb,X_test_tfidf2,y_pred_tfidf2_nb)

Accurcy : 1.0
F(1): 1.0
Precision : 1.0
Recall : 1.0


In [72]:
# Compile all of the error metrics into a dataframe for comparison
results_tf = pd.DataFrame(list(zip(me1, me2, me3, me4,me5, me6, me7, me8,me9, me10, me11, me12)))
results_tf = results_tf.set_index([['Accuracy', 'Precision', 'Recall', 'F1 Score']])
results_tf.columns = ['LR1-CV', 'LR2-CV', 'NB1-MN-CV', 'NB2-MN-CV','NB1-BR-CV', 'NB2-BR-CV','LR1-TFIDF', 'LR2-TFIDF', 'NB1-MN-TFIDF', 'NB2-MN-TFIDF','NB1-BR-TFIDF', 'NB2-BR-TFIDF']
results_tf
#'Accuracy', 'F1 Score', 'Recall', 'Precision'


,LR1-CV,LR2-CV,NB1-MN-CV,NB2-MN-CV,NB1-BR-CV,NB2-BR-CV,LR1-TFIDF,LR2-TFIDF,NB1-MN-TFIDF,NB2-MN-TFIDF,NB1-BR-TFIDF,NB2-BR-TFIDF
Accuracy,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Precision,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Recall,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
F1 Score,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
